In [ ]:
!pip install -q -U transformers datasets peft bitsandbytes accelerate trl scipy sentencepiece

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-jewa3bxy/unsloth_353b00060b504d62856910ad2f4f9638
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-jewa3bxy/unsloth_353b00060b504d62856910ad2f4f9638
  Resolved https://github.com/unslothai/unsloth.git to commit f3e95372cdff1b665110868a97816eac571df723
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 135.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.4/284.4 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 134.6 MB/s eta 0:00:

In [ ]:
from google.colab import files
import os

uploaded = files.upload()


Saving wuzzuf_train_llama.jsonl to wuzzuf_train_llama.jsonl
Saving wuzzuf_validation_llama.jsonl to wuzzuf_validation_llama.jsonl


In [ ]:
from unsloth import FastLanguageModel
import torch

dtype = None
load_in_4bit = True


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
max_seq_length = 2048

In [ ]:
!watch -n 1 nvidia-smi

>

In [ ]:
model_name = "unsloth/Llama-3.2-3B-Instruct"

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
from unsloth import FastLanguageModel

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

In [ ]:
from datasets import load_dataset

train_dataset = load_dataset('json', data_files='wuzzuf_train_llama.jsonl', split='train')
val_dataset = load_dataset('json', data_files='wuzzuf_validation_llama.jsonl', split='train')


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
train_dataset[0]

{'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are an expert HR assistant specialized in creating professional job postings. When provided with minimal input (job title, career level, location, department, company, and optional key skills), your task is to generate two distinct, mandatory outputs:\n1. A complete, engaging, and professional job posting in natural language format.\n2. Structured JSON data for database storage.\nYou must follow this format precisely: generate the natural language posting first, followed by the unique separator: ---STRUCTURED-DATA--- and then the JSON output. The JSON must contain the following exact 10 keys: title, company, job_type, work_setting, location, experience_needed, career_level, education_level, job_categories, and skills. All ten fields must be completed, using the inferred value or \'N/A\' if the data point is truly unknown. The value for the skills key must be a single, comma-separated string.<|eot_id|><|start_h

In [ ]:
from datasets import load_dataset
from unsloth.chat_templates import get_chat_template

In [ ]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3",
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"},
)


In [ ]:
def formatting_prompts_func(examples):
    instructions=examples["instruction"]
    inputs=examples["input"]
    outputs=examples["output"]
    texts=[]

    for instruction, input, output in zip(instructions, inputs, outputs):
        conversation = [
            {"from": "system", "value": instruction},
            {"from": "human", "value": input },
            {"from": "gpt", "value": output},
        ]
        text = tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=False)
        texts.append(text)

    return { "text" : texts }

In [ ]:
train_df = load_dataset('json', data_files='wuzzuf_train.jsonl', split='train')
train_df = train_df.map(formatting_prompts_func, batched = True)



Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/4533 [00:00<?, ? examples/s]

In [ ]:
shuffled_dataset = train_df.shuffle(seed=42)

In [ ]:
sampled_pool = shuffled_dataset.select(range(800))

In [ ]:
split_data = sampled_pool.train_test_split(test_size=0.1, seed=42)

In [ ]:
train_dataset = split_data['train']
eval_dataset = split_data['test']

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import FastLanguageModel

In [ ]:
training_args = TrainingArguments(
    output_dir              = "./output",

    per_device_train_batch_size = 2,
    per_device_eval_batch_size  = 2,
    gradient_accumulation_steps = 4,

    num_train_epochs = 1,

    learning_rate = 2e-4,
    warmup_ratio  = 0.03,

    eval_strategy = "steps",
    eval_steps    = 20,
    save_strategy = "epoch",
    logging_steps = 5,

    optim = "adamw_8bit",
    weight_decay = 0.01,
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),

    report_to = "none",
    seed = 3407,
)

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False,
    args = training_args,
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/720 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/80 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 720 | Num Epochs = 1 | Total steps = 90
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
20,0.927900,0.925645
40,0.881900,0.854165
60,0.738500,0.833373
80,0.786400,0.823916


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


In [ ]:
eval_results = trainer.evaluate()

print("\n" + "="*80)
print("📈 FINAL RESULTS")
print("="*80)
print(f"   Training loss: {trainer_stats.training_loss:.4f}")
print(f"   Validation loss: {eval_results['eval_loss']:.4f}")
print("="*80)



📈 FINAL RESULTS
   Training loss: 0.9289
   Validation loss: 0.8221


In [ ]:
import json
training_info = {
    "model": model_name,
    "training_loss": float(trainer_stats.training_loss),
    "validation_loss": float(eval_results['eval_loss']),
    "training_time_minutes": 15 / 60,
    "epochs": training_args.num_train_epochs,
    "total_steps": trainer_stats.global_step,
    "lora_rank": 16,
    "effective_batch_size": training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps,
}

with open("./lora_model_final/training_info.json", "w") as f:
    json.dump(training_info, f, indent=2)



💾 Saving model...
✅ Model saved to: ./lora_model_final
✅ Training info saved


In [ ]:
FastLanguageModel.for_inference(model)

test_system_prompt = """You are an expert HR assistant specialized in creating professional job postings. When provided with minimal input (job title, career level, location, department, company, and optional key skills), your task is to generate two distinct, mandatory outputs:
1. A complete, engaging, and professional job posting in natural language format.
2. Structured JSON data for database storage.
You must follow this format precisely: generate the natural language posting first, followed by the unique separator: ---STRUCTURED-DATA--- and then the JSON output. The JSON must contain the following exact 10 keys: title, company, job_type, work_setting, location, experience_needed, career_level, education_level, job_categories, and skills. All ten fields must be completed, using the inferred value or 'N/A' if the data point is truly unknown. The value for the skills key must be a single, comma-separated string."""

conversation = [
    {"role": "system", "content": test_system_prompt},
    {"role": "user", "content": "Job Title: Quantum Computing Analyst\nCareer Level: Senior\nLocation: Berlin, Germany\nDepartment: R&D\nCompany: Qubit Solutions"},
    {"role": "assistant", "content": ""},
]

prompt_string = tokenizer.apply_chat_template(
    conversation,
    tokenize=False,
    add_generation_prompt=True
)

inputs = tokenizer(
    prompt_string,
    return_tensors="pt"
)

inputs = {k: v.to("cuda") for k, v in inputs.items()}

outputs = model.generate(
    **inputs,
    max_new_tokens=1500,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id,
)
text = tokenizer.decode(outputs[0])

print("\n--- Generated Output (Checklist) ---")
generated_text = text.split('<|start_header_id|>assistant<|end_header_id|>')[-1].strip()

print(generated_text)
print("-----------------------------------")


--- Generated Output (Checklist) ---
We are Looking for a talented**Quantum Computing Analyst at Qubit Solutions**

### Key Details
**Location:** Berlin, Germany
**Job Type:** Full Time
**Work Setting:** Hybrid
**Career Level:** Senior
**Education Required:** Bachelor's Degree
**Experience Needed:** More Than 4 Years


**What You'll Do & The Impact You'll Have**

•  We are seeking a highly motivated Quantum Computing Analyst to join our team at Qubit Solutions. This is a unique opportunity to work on cutting-edge projects and contribute to the development of quantum computing technology.
•  As a Quantum Computing Analyst, you will be responsible for:
•  Analyzing and optimizing quantum algorithms for various applications.
•  Designing and implementing quantum circuits using quantum programming languages.
•  Developing and testing quantum software for simulation and optimization tasks.
•  Collaborating with our team of quantum computing experts to advance our research and development e

In [ ]:
from huggingface_hub import login, HfApi

print("🔑 Please login to Hugging Face:")
login()


🔑 Please login to Hugging Face:


In [ ]:
repo_name = "farahelmashad/llama32-3b-finetuned-lora-new"
print(f"\n📤 Creating repository: {repo_name}")


📤 Creating repository: farahelmashad/llama32-3b-finetuned-lora-new


In [ ]:
api = HfApi()
api.create_repo(repo_id=repo_name, private=False, exist_ok=True)

RepoUrl('https://huggingface.co/farahelmashad/llama32-3b-finetuned-lora-new', endpoint='https://huggingface.co', repo_type='model', repo_id='farahelmashad/llama32-3b-finetuned-lora-new')

In [ ]:


print(f"📤 Uploading model files...")

api.upload_folder(
    folder_path="./lora_model_final",
    repo_id=repo_name,
    repo_type="model",
)

print(f"\n✅ Model uploaded successfully!")
print(f"🔗 View at: https://huggingface.co/{repo_name}")


📤 Uploading model files...


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...odel_final/tokenizer.json: 100%|##########| 17.2MB / 17.2MB            

  ...adapter_model.safetensors:   1%|1         |  525kB / 48.7MB            


✅ Model uploaded successfully!
🔗 View at: https://huggingface.co/farahelmashad/llama32-3b-finetuned-lora-new


In [ ]:
repo_name = "farahelmashad/llama32-3b-job-posting-merged-new"
print("\n📤 Uploading MERGED 16-bit model...")
try:
    model.push_to_hub_merged(
        repo_name,
        tokenizer,
        save_method="merged_16bit"
    )
    print(f"✅ Merged 16-bit Model uploaded to: https://huggingface.co/{repo_name}")
except Exception as e:
    print(f"❌ Failed to upload merged 16-bit model. Try reducing the max memory usage. Error: {e}")


📤 Uploading MERGED 16-bit model...


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...merged-new/tokenizer.json:  48%|####8     | 8.33MB / 17.2MB            

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


...

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...0002-of-00002.safetensors:   0%|          | 4.26MB / 1.46GB            

Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [04:16<00:00, 128.34s/it]


Unsloth: Merge process complete. Saved to `/content/farahelmashad/llama32-3b-job-posting-merged-new`
✅ Merged 16-bit Model uploaded to: https://huggingface.co/farahelmashad/llama32-3b-job-posting-merged-new


In [ ]:
repo_name_gguf = repo_name + "-GGUF"
print(f"\n📤 Uploading GGUF (Q4_K_M) model to a separate repo...")
try:
    model.push_to_hub_gguf(
        repo_name_gguf,

        tokenizer,
        quantization_method="q4_k_m"
    )
    print(f"✅ GGUF Model uploaded to: https://huggingface.co/{repo_name_gguf}")
except Exception as e:
    print(f"❌ Failed to upload GGUF model. Error: {e}")

NameError: name 'repo_name' is not defined

In [ ]:
repo_name_lora = repo_name + "-LoRA-Adapter"
print("\n📤 Uploading LoRA Adapters (original small files)...")
model.push_to_hub(repo_name_lora, tokenizer)
print(f"✅ LoRA Adapters uploaded to: https://huggingface.co/{repo_name_lora}")